Check against the Standard Summary Report. There is a federal website that can do similar reports: http://tinyurl.com/nkjcno5
    
    

In [102]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type
import pandas as pd
import numpy as np
from ambry import library


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition

source = p.select("SELECT * FROM lar WHERE county_code = 73").pandas


Figure out what the respondent_id is for Bank of America

In [87]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = source[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('BANK OF AMERICA')][['respondent_id','respondent_name']].drop_duplicates()

,respondent_id,respondent_name
4398,0000012773,BANK OF AMERICAN FORK
17927,0000014740,FIRST NATIONAL BANK OF AMERICA
263517,0000480228,"BANK OF AMERICA, N.A."


In [106]:
sd_msa = lib.util.tract_to_msa()
df = source.copy()
add_race_eth(df)
add_purpose_type(df) # Combines loan_purpose and loan_type in the same way as the summary report


Compute the totals for the count and load amount for the first section of the Standard Summary Report for the row/columns "Load Purpose and Type / Total Applications "

In [107]:

x = df[(df.msa_md  == 41740) & ( df.respondent_id == '0000480228' )  ][['purpose_type','action_type','loan_amount']]
x.groupby('purpose_type').count()[['action_type']]

,action_type
purpose_type,
improvement,103
purchase-conventional,1022
purchase-govt,550
refinance,8271


In [108]:
x.groupby('purpose_type').sum()[['loan_amount']]

,loan_amount
purpose_type,
improvement,32417
purchase-conventional,484014
purchase-govt,174186
refinance,2783718


In [109]:

x = df[(df.msa_md  == 41740) & ( df.respondent_id == '0000480228' ) & ( df.action_type == 1 ) ]\
[['purpose_type','action_type','loan_amount']]
x.groupby('purpose_type').count()[['action_type']]

,action_type
purpose_type,
improvement,52
purchase-conventional,622
purchase-govt,243
refinance,5439


In [110]:
source[(source.msa_md  == 41740) & ( source.respondent_id == '0000480228' ) ].id.count()

9946